In [78]:
# Install dependencies
%pip install ultralytics
%pip install opencv-python
%pip install simple_pid
%pip install numpy
%pip install pyserial
%pip install cv2-enumerate-cameras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Configurations
import cv2
import math
import serial
import time
import numpy as np
from ultralytics import YOLO
from simple_pid import PID
from cv2_enumerate_cameras import enumerate_cameras

# Camera configuration
CAMERA_INDEX = 701                 # Index of the camera to be used (custom value; check with enumerate_cameras)
FRAME_DELAY = 0.2                  # Delay between frames in seconds for processing

# Geometry setup
CAMERA_DISTANCE = 20               # Distance between two cameras (used for triangulation)
DISTANCE_UNIT = "cm"               # Unit of measurement for camera distance

# YOLO model setup
MODEL = YOLO('yolov8n.pt')         # Load YOLOv8 nano model for object detection
CLASS_NAMES = MODEL.names          # List of all class labels in the model
DESIRED_CLASSES = ["sports ball"]  # Classes we want to detect (only track sports balls)

# PID control parameters
PID_KP = 0.05                      # PID Proportional gain
PID_KI = 0.01                      # PID Integral gain
PID_KD = 0.05                      # PID Derivative gain
PID_OUTPUT_LIMITS = (-10, 10)      # Output limits for PID correction to smooth servo movement

# Servo configuration
SERVO_MIN = 0                      # Minimum angle a servo can rotate to
SERVO_MAX = 180                    # Maximum angle a servo can rotate to
SERVO_INIT = {                     # Initial servo positions (centered)
    'x1': 90, 
    'y1': 90, 
    'x2': 90, 
    'y2': 90
}

# Arduino serial communication
ARDUINO_PORT = 'COM4'              # Serial port where Arduino is connected
ARDUINO_BAUDRATE = 9600            # Communication speed between Python and Arduino

In [ ]:
# CV2 and camera settings

# Initialize camera
def init_camera(index=CAMERA_INDEX, verbose=True):
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        raise RuntimeError(f"Error: Could not open webcam {index}.")
    if verbose:
        print(f"Camera {index} opened successfully.")
    return cap


# Read current frame
def read_frame(cap):
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to capture image from webcam.")
    return frame


# Calculate frame center
def calculate_center(frame):
    height, width = frame.shape[:2]
    x_center, y_center = width / 2, height / 2
    return x_center, y_center


# Check if class is in desired classes
def valid_class(box, class_names=CLASS_NAMES, desired_classes=DESIRED_CLASSES):
    return class_names[int(box.cls)] in desired_classes


# Draw bounding box
def draw_bounding_box(frame, box, class_names=CLASS_NAMES, verbose=True):
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
    
    cls = int(box.cls[0])
    confidence = math.ceil((box.conf[0] * 100)) / 100
    
    org = [x1, y1]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (255, 0, 0)
    thickness = 2
    cv2.putText(frame, class_names[cls], org, font, font_scale, color, thickness)
    
    if verbose:
        print(f"Class: {class_names[cls]}, Confidence: {confidence}")

In [91]:
# Calculations

# Calculate balloon coordinates
def triangulation(angles: dict, d=CAMERA_DISTANCE, verbose=True):
    alpha = np.radians(angles['x1'])
    beta = np.radians(angles['x2'])
    gamma = np.radians(angles['y1'])
    delta = np.radians(angles['y2'])
    
    if (np.sin(alpha + beta)) == 0:
        raise ValueError("Invalid angles for triangulation.")
    
    n = d * np.sin(beta) / np.sin(alpha + beta)
    m = d * np.sin(alpha) / np.sin(alpha + beta)

    x1 = n * np.cos(alpha)
    y1 = n * np.sin(alpha)
    z1 = n * np.tan(gamma)
    
    x2 = d - (m * np.cos(beta))
    y2 = m * np.sin(beta)
    z2 = m * np.tan(delta)

    x = (x1 + x2) / 2
    y = (y1 + y2) / 2
    z = (z1 + z2) / 2
    
    if verbose:
        print(f"Balloon position: ({x:.2f}, {y:.2f}, {z:.2f})")
    
    return np.array([x, y, z])


# Calculate distance to camera
def get_distance(camera, balloon_position, d=CAMERA_DISTANCE, verbose=True):
    if camera == 'C1':
        camera_position = np.array([0, 0, 0])
    elif camera == 'C2':
        camera_position = np.array([d, 0, 0])
    else:
        raise ValueError("Invalid camera ID. Use 'C1' or 'C2'.")
    
    distance = np.linalg.norm(balloon_position - camera_position)
    
    if verbose:
        print(f"{camera} Distance: {distance:.2f} {DISTANCE_UNIT}")
    
    return distance

In [ ]:
# PID controller

# Initialize PID controller
def init_pid(x_center, y_center, verbose=True):
    pid_pan = PID(Kp=PID_KP, Ki=PID_KI, Kd=PID_KD, setpoint=x_center)
    pid_pan.output_limits = PID_OUTPUT_LIMITS
    pid_tilt = PID(Kp=PID_KP, Ki=PID_KI, Kd=PID_KD, setpoint=y_center)
    pid_tilt.output_limits = PID_OUTPUT_LIMITS
    if verbose:
        print("PID controller initialized.")
    return pid_pan, pid_tilt


# Calculate corrections using PID controller
def calculate_corrections(box, pid_pan, pid_tilt, verbose=True):
    x1, y1, x2, y2 = box.xyxy[0]
    x_box = float((x1 + x2) / 2)
    y_box = float((y1 + y2) / 2)
    pan, tilt = pid_pan(x_box), pid_tilt(y_box)
    if verbose:
        print(f"Box: ({x_box}, {y_box})")
        print(f"Pan: {pan}, Tilt: {tilt}")
    return pan, tilt


# Update servo angle
def update_angle(servo_angle, correction):
    if correction > 10:
        correction = 10
    elif correction < -10:
        correction = -10
    return max(SERVO_MIN, min(SERVO_MAX, math.ceil(servo_angle + correction)))

In [93]:
# Arduino

# Initialize serial connection to arduino
def init_arduino(*, port=ARDUINO_PORT, baudrate=ARDUINO_BAUDRATE, initial_angles=SERVO_INIT, verbose=True):
    try:
        arduino = serial.Serial(port, baudrate)
        time.sleep(2)
        if verbose:
            print(f"Connected to Arduino on {port}.")
        move_servos(arduino, initial_angles)
        return arduino
    except serial.SerialException as e:
        raise RuntimeError(f"Could not connect to Arduino on {port}: {e}")


# Move servos
def move_servos(arduino, angles: dict, verbose=True):
    try:
        command = f"{angles['x1']},{angles['y1']},{angles['x2']},{angles['y2']}\n"
        arduino.write(command.encode())
        arduino.flush()
        if verbose:
            print(f"C1 → X: {angles['x1']:.1f}°, Y: {angles['y1']:.1f}°")
            print(f"C2 → X: {angles['x2']:.1f}°, Y: {angles['y2']:.1f}°")
            
    except serial.SerialException as e:
        print(f"Serial communication error: {e}")

In [99]:
# Print functions

# Print divider
def print_div(text):
    print("\n" + f" {text} ".center(100, '-') + "\n")


# Print available cameras
def print_cameras():
    for camera_info in enumerate_cameras():
        print(f'{camera_info.index}: {camera_info.name}')


# Print final information
def final_prints(frame_count, x_center, y_center, balloon_position, dist_c1, dist_c2, servo_angles):
    x, y, z = balloon_position
    print(f"Total frames: {frame_count}")
    print(f"Detected classes: {DESIRED_CLASSES}")
    print(f"Frame center: ({x_center}, {y_center})")
    print(f"FPS: {(1/FRAME_DELAY):.2f}")
    print(f"Final balloon position: ({x:.2f}, {y:.2f}, {z:.2f})")
    print(f"Final C1 distance: {dist_c1:.2f} {DISTANCE_UNIT}")
    print(f"Final C2 distance: {dist_c2:.2f} {DISTANCE_UNIT}")
    print(f"Servo angles: {servo_angles}\n")

In [100]:
# Main program
print_div("Available cameras")
print_cameras()

print_div("Initial settings")
frame_count = 0
servo_angles = SERVO_INIT
arduino = init_arduino()
cap = init_camera()
frame = read_frame(cap)
x_center, y_center = calculate_center(frame)
pid_pan, pid_tilt = init_pid(x_center, y_center)

print_div("Program started")
try:
    while True:
        x1_correction, y1_correction = 0, 0
        frame = read_frame(cap)
        results = MODEL(frame, stream=True)
        for r in results:
            if not r.boxes:
                continue
            for box in r.boxes:
                if valid_class(box):
                    draw_bounding_box(frame, box)
                    x1_correction, y1_correction = calculate_corrections(box, pid_pan, pid_tilt)
        
        print(f"Frame number: {frame_count}")
        balloon_position = triangulation(servo_angles)
        dist_c1 = get_distance('C1', balloon_position)
        dist_c2 = get_distance('C2', balloon_position)
        print(f"Servo angles: {servo_angles}")
        
        if x1_correction != 0 or y1_correction != 0:
            y1_correction *= -1 # type: ignore
            servo_angles['x1'] = update_angle(servo_angles['x1'], x1_correction)
            servo_angles['y1'] = update_angle(servo_angles['y1'], y1_correction)
            move_servos(arduino, servo_angles)
        
        cv2.imshow("Image", frame)
        if cv2.waitKey(1) == ord('q'):
            break
        frame_count += 1
        time.sleep(FRAME_DELAY)
finally:
    print_div("Program finished")
    final_prints(frame_count, x_center, y_center, balloon_position, dist_c1, dist_c2, servo_angles)
    cap.release()
    cv2.destroyAllWindows()
    arduino.close()


---------------------------------------- Available cameras -----------------------------------------

1400: HD User Facing
1401: Logi C270 HD WebCam
700: HD User Facing
701: Logi C270 HD WebCam

----------------------------------------- Initial settings -----------------------------------------

Connected to Arduino on COM4.
C1 → X: 90.0°, Y: 90.0°
C2 → X: 90.0°, Y: 90.0°
Camera 701 opened successfully.
PID controller initialized.

----------------------------------------- Program started ------------------------------------------


0: 480x640 1 cup, 1 potted plant, 1 dining table, 1 book, 1 vase, 118.7ms
Speed: 3.2ms preprocess, 118.7ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)
Frame number: 0
Balloon position: (10.00, 163312393531953696.00, 2667093788113571165172990803443712.00)
C1 Distance: 2667093788113571165172990803443712.00 cm
C2 Distance: 2667093788113571165172990803443712.00 cm
Servo angles: {'x1': 90, 'y1': 90, 'x2': 90, 'y2': 90}

0: 480x640 1 cup, 1 